## 3 TRY
하라는 대로 3번 반복 .
2번째에서야 로직을 거의 이해했고
3번째에 대하여 그래서 안해도되지않나 싶었는데
그 정돈 되야 안보고 로직을 대충 아니까 세세한 문법과 세세한 코드를 익힌다. 어 이렇게 하면 되지 않을까 하는 생각이 든다
반복을 했으니까. 그리고 거기서 응용 해서 안보고 최대한 짜도록 노력,
처음엔 얼마나 무의식적으로 했는 지 알게 될것이다 관성에 의해, 의식하지 않는다면


In [ ]:
import pandas as pd, numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import *
print('TF version',tf.__version__)

In [ ]:
assert(1==1)

In [ ]:
# USE MULTIPLE GPUS
try:
    if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0 :
        strategy = tf.distribute.get_strategy()
        print('single strategy')
    else:
        strategy = tf.distribute.MirroredStrategy()
        print('Multiple strategy')
except KeyError:
    # keyerror 만 발생한다고 가정
    print('CUDA_VISIBLE_DEVICES KEY NOT EXISTS ')
    strategy = tf.distribute.get_strategy()
    print('single strategy')

In [ ]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

In [ ]:
LOAD_PATH = '../input/feedback-prize-2021'

In [ ]:
train = pd.read_csv(f'{LOAD_PATH}/train.csv')

In [ ]:
IDS = train['id'].unique()

In [ ]:
print(f'There are {len(IDS)} train texts')

In [ ]:
print('The train labels are: ')
train.discourse_type.unique()

# Tokenize Train

In [ ]:
DOWNLOADED_MODEL_PATH = '../input/tf-longformer-v12'

> path 만 주었을 때 Tokenizer 를 읽는다

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH)

In [ ]:
tokenizer

In [ ]:
MAX_LEN = 1024

In [ ]:
train_tokens = np.zeros((len(IDS),MAX_LEN), dtype='int32')

In [ ]:
train_attention = np.zeros((len(IDS),MAX_LEN),dtype='int32')

In [ ]:
lead_b = np.zeros((len(IDS),MAX_LEN))

In [ ]:
np.array(targets_b).shape  # 각 클래스별로 beginning 은 또 달라

In [ ]:
# THE 14 CLASSES FOR NER

lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN), dtype='int32')

position_b = np.zeros((len(IDS), MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))


claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b =np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))


counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))


rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

#HELPER VARIABLES
train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i , claim_i, conclusion_i, counterclaim_i, rebuttal_i]

target_map = {'Lead' : 0 , 'Position':1,'Evidence':2, 'Claim':3, 'Concluding Statement':4,
              'Counterclaim':5, 'Rebuttal':6}


In [ ]:
# WE ASSUME DATAFRAME IS ASCENDING WHICH IT IS

In [ ]:
assert(np.sum(train.groupby('id')['discourse_start'].diff()<=0)==0)

In [ ]:
LOAD_TOKENS_FROM = None

In [ ]:
tokenizer.encode_plus(txt)

In [ ]:
train_lens = [ ]

In [ ]:
tokens = tokenizer.encode_plus(txt,  max_length=MAX_LEN, padding='max_length',
                                 truncation=True , return_offsets_mapping=True)

In [ ]:

# FOR LOOP THROUGH EACH TRAIN TEXT
for id_num in range(len(IDS)):
    if LOAD_TOKENS_FROM: break
    if id_num%100==0: print(id_num, ', ', end='')

    # READ TRAIN TEXT , TOKENIZE , AND SAVE IN TOKEN ARRAYS
    n = IDS[id_num]
    name = f'../input/feedback-prize-2021/train/{n}.txt'
    txt = open(name,'r').read()
    train_lens.append(len(txt.split()))
     # truncation 은 엄격하게 문장을 자른다
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN , padding='max_length'
                       , truncation=True, return_offsets_mapping=True)
    train_tokens[id_num,] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']


    offsets = tokens['offset_mapping']
    offset_index = 0
        # input id 로 tokenizing 이 됐으니까 offset_index를 가지고
    # 원래의 어떤 단어가 어디까지 tokenizing이 된거고 그 어떤 단어가 뭔지 볼려고 하는 거지
    for index, row in df.iterrows():
        if offset_index > len(offsets) -1:
            break
        a =  row.discourse_start
        b =  row.discourse_end
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]

        beginning = True
        while b>c:# 어쨌든 해당 offset이 row에 있는 지는 check를 해야 한다
            if (c>=a) & (b>=d): # offset의 시작점이 row 안에 있느냐
                # offset의 끝점이 row 안에 있느냐
                k = target_map[row.discourse_type] # target map에서 가져와
                if beginning:  # 시작점이면 , 알고리즘 뿐 아니라 자료구조자체도 좀 복잡하다
                    # offset이 token화 된거니까 offset_index를 찾는다
                    # k번째 클래스에 df의 id_num 번째의 offset_index를 target처리하면
                    # 학습 시 K번째 class 의 id_num의 offset_index로 학습한다
                    # 어차피 train id의 전체 묶음이 IDS이므로 id_num에 입력하면 된다
                    targets_b[k][id_num][offset_index] = 1
                    beginning =False
                else:
                    targets_i[k][id_num][offset_index] = 1
            offset_index +=1
            if offset_index > len(offsets)-1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]

        # FIND TARGETS IN TEXT AND SAVE IN TARGET ARRAYS

In [ ]:
targets = np.zeros((len(IDS),MAX_LEN,15),dtype='int32')

In [ ]:
targets

In [ ]:
np.array(targets_b).shape

In [ ]:
np.array(targets_i).shape

In [ ]:
np.array(targets).shape

In [189]:
for k in range(7):
    # like .iloc indexing
    try:
        targets[:,:,2*k] = targets_b[k]
        targets[:,:,2*k+1] = targets_i[k]
        raise ValueError # 일부러 에러 발생시키는 코드
    except ValueError:  # 발생된 에러에 대한 예외처리
                        # raise 하지 않고 예외처리에만 에러를 입력하면
                        # except로 빠지지 않는다
        print(np.array(targets[:,:,2*k].shape),
              np.array(targets_b[k].shape),
              np.array(targets_i[k].shape),k)

[15594  1024] [15594  1024] [15594  1024] 0
[15594  1024] [15594  1024] [15594  1024] 1
[15594  1024] [15594  1024] [15594  1024] 2
[15594  1024] [15594  1024] [15594  1024] 3
[15594  1024] [15594  1024] [15594  1024] 4
[15594  1024] [15594  1024] [15594  1024] 5
[15594  1024] [15594  1024] [15594  1024] 6


In [ ]:
# targets 의 마지막 axis 에 대하여 np.max값을 취한다
targets[:,:,14] = 1-np.max(targets,axis=-1)

In [207]:
def build_model():
    # layer를 할당해서 multiple 하는 방식으로 build model 한다
    tokens = tf.keras.layers.Input(shape= (MAX_LEN,) ,name='tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape = (MAX_LEN,), name='attention',dtype=tf.int32)

    config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json')
    # backbone is tensorflow tf-longformer-v12 (transformer 모델)
    # tf_model.h5 is real model
    # bigbird is another recent SOTA transformers
    backbone =TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH+'/tf_model.h5', config=config )

    x = backbone(tokens ,attention_mask=attention )
    # what is x[0]?
    x = tf.keras.layers.Dense(256, activation='relu')(x[0])
    x = tf.keras.layers.Dense(15, activation='softmax', dtype='float32')(x)

    #like lazy loading
    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr =1e-4)  ,
                   loss = [tf.keras.losses.CategoricalCrossentropy()],
                   metrics = [tf.keras.metrics.CategoricalAccuracy()])

    return model

In [208]:
with strategy.scope():
    model = build_model()

loading configuration file ../input/tf-longformer-v12/config.json
Model config LongformerConfig {
  "_name_or_path": "../input/tf-longformer-v12/config.json",
  "architectures": [
    "LongformerModel"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sep_token_id": 2,
  "transformers_version": "4.16.2",
  "type_vocab_size": 1,
  "use_cache": tru

In [203]:
EPOCHS = 5
BATCH_SIZE = 4
LRS=  [0.25e-4, 0.25e-4, 0.25e-4, 0.25e-4 , 0.25e-5]
def lrfn(epoch):
    return LRS[epoch]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [210]:
# TRAIN VALID SPLIT 90%, 10%
np.random.seed(42)

In [211]:
train_idx = np.random.choice(np.arange(len(IDS)), int(0.9*len(IDS)) , replace=False)


array([[    0, 23314,   194, ...,     1,     1,     1],
       [    0, 23314,   427, ...,     1,     1,     1],
       [    0, 46022,  1672, ...,     1,     1,     1],
       ...,
       [    0,  1121,    42, ...,    79,  1447,     2],
       [    0,  1121,    22, ...,     1,     1,     1],
       [    0,  8275,    47, ...,     1,     1,     1]], dtype=int32)

In [213]:
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)

In [214]:
np.random.seed(None)

In [215]:
print('Train Size', len(train_idx) , ', Valid size', len(valid_idx))

Train Size 14034 , Valid size 1560


In [216]:
# LOAD MODEL
if LOAD_MODEL_FROM:
    model.load_weight(f'{LOAD_MODEL_FROM}/long_v{VER}.h5')

NameError: name 'LOAD_MODEL_FROM' is not defined

array([10360, 10224,  8720, ..., 13395,  4944,   966])

In [227]:
train_tokens[train_idx]

array([[    0, 23314,   194, ...,     1,     1,     1],
       [    0, 23314,   427, ...,     1,     1,     1],
       [    0, 46022,  1672, ...,     1,     1,     1],
       ...,
       [    0,  1121,    42, ...,    79,  1447,     2],
       [    0,  1121,    22, ...,     1,     1,     1],
       [    0,  8275,    47, ...,     1,     1,     1]], dtype=int32)

In [229]:
# OR TRAIN MODEL
model.fit( x = [train_tokens[train_idx,] , train_attention[train_idx]],
           y = targets[train_idx,]
         , validation_data = ([ train_tokens[valid_idx] ,train_tokens[valid_idx]] ,
                                targets[valid_idx,]),
        callbacks = [lr_callback],
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        verbose= 2 )
model.save_weights(f'long_v{VER}.h5')


Epoch 1: LearningRateScheduler setting learning rate to 2.5e-05.
Epoch 1/5


2022-02-20 10:28:56.195887: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2218] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer
2022-02-20 10:28:56.212352: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2218] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer
2022-02-20 10:28:56.227675: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2218] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer


2022-02-20 10:29:56.590152: W tensorflow/core/grappler/optimizers/auto_mixed_precision.cc:2218] No (suitable) GPUs detected, skipping auto_mixed_precision graph optimizer


KeyboardInterrupt: 

In [ ]:
p = model.predict([train_tokens[valid_idx,], train_attention[valid_idx,]],
                  batch_size=16, verbose=2)
print("OOF predictions shape:", p.shape)
oof_preds = np.argmax(p, axis=-1)
# OOF predictino shape: (1560, 1024, 15) (len(valid_idx), MAX_LEN, class)

In [ ]:
targets[:,:,2*1] = targets_b[1]

In [ ]:
target_map_rev = { 0:'Lead', 1:'Position', 2:'Evidence' , 3:'Claim', 4:'Concluding Statement', 5:'Counterclaim', 6:'Rebuttal', 7:'blank'}

In [270]:
def get_preds(  dataset='train', verbose=True , text_ids=IDS[valid_idx],   preds = oof_preds):
    all_predictions =[]

    for id_num in range(len(preds)):
        # GET ID
        if (id_num%100 ==0) & (verbose):
            print(id_num, ', ', end='')
        n = text_ids[id_num]

        # GET TOKEN POSITION IN CHARS
        name = f'../input/feedback-prize-2021/{dataset}/{n}.txt'
        txt = open(name, 'r').read()
        tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                        truncation=True , return_offsets_mapping=True)
        off =  tokens['offset_mapping']

        # GET WORD POSITIONS IN CHARS
        w = []
        blank = True

        for i in range(len(txt)):
            if (txt[i]!=' ') & (txt[i]!= '\n')&(txt[i]!='\xa0')&(txt[i]!='\x85')&(blank==True):
                w.append(i)
                blank=False
            elif (txt[i] == ' ')|(txt[i] =='\n')|(txt[i]=='\xa0')|(txt[i]=='\x85'):
                blank=True
        w.append(1e6)

        #MAPPING FROM TOKENS TO WORDS
        word_map = -1 * np.ones(MAX_LEN, dtype='int32')
        w_i = 0
        for i in range(len(off)):
            if off[i][1] == 0:
                continue
            while off[i][0]>=w[w_i+1]: # offset 시작점이 w_i안에 있다면
                                        # 동일한 word_index로 취급한다
                                        # word[word_idx+=1]
                w_i+=1
                word_map[i] = w_i # word_map 의 i 번째는 w_i 값 word 가 blank가 아닌 값이다 즉 word_index

        # convert token predictions into word labels
        ### KEY: ###
        # 0: LEAD_B, 1: LEAD_I
        # 2: POSITION_B , 3: POSITION_I
        # 4: EVIDENCE_B , 5: EVIDENCE_I
        # 6: CLAIM_B , 7:CLAIM_I
        # 8: CONCLUSION_B , 9: CONCLUSION_I
        # 10: COUNTERCLAIM_B, 11: COUNTERCLAIM_I
        # 12: REBUTTAL_B , 13: REBUTTAL_I
        # 14: NOTHING i.e. 0
        ### NOTE THESE VALUES ARE DIVIDED BY 2 IN NEXT CODE LINE
        pred = preds[id_num,]/2.0

        i = 0
        while i<MAX_LEN:
            prediction=[]
            start = pred[i]
            if start in [0,1,2,3,4,5,6,7]:
                prediction.append(word_map[i])
                i += 1
                if i>=MAX_LEN:
                    break
                while pred[i] == start+0.5:
                    if not word_map[i] in prediction:
                        prediction.append(word_map[i])
                    i+=1
                    if i>=MAX_LEN:
                        break
            else:
                i += 1
            prediction = [x for x in prediction if x!=-1]
            if len(prediction) >4:
                all_predictions.append( (n, target_map_rev[int(start)],
                                         ' '.join([str(x) for x in prediction])))
    # MAKE DATAFRAME
    df = pd.DataFrame(all_predictions)
    df.columns = ['id','class','predictionstring']

    return df

NameError: name 'oof_preds' is not defined

In [271]:
oof = get_preds(dataset='train' , verbose=True, text_ids=IDS[valid_idx])
oof.head()

NameError: name 'get_preds' is not defined

In [272]:
print('The following classes are present in oof preds:')

The following classes are present in oof preds:


In [273]:
oof['class'].unique()

NameError: name 'oof' is not defined

### Compute Validation Metric
the following code if from Rob Mulla's excellent notebook [here](https://www.kaggle.com/robikscube/student-writing-competition-twitch-stream?scriptVersionId=83303421). Our LongFormer single fold model achieves CV score 0.633! Hooray!

In [ ]:
# CODE FROM : Rob Mulla @robikscube

def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives

    :param row:
    :return:
    """
    #row[['id','class','predictionstring' , 'predictionstring_pred','predictionstring_gt']]\
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))

    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)

    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter /len_pred


    return [overlap_1, overlap2]

    # overlap되는 길이를 측정.


In [239]:
# VALID DATAFRAME
valid  = train.loc[train['id'].isin(IDS[valid_idx])]

In [ ]:
f1s =[]
CLASSES = oof['class'].unique()


In [276]:
def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
    Student Writing Completion

    Uses the steps in the evaluation page here:

    :param pred_df:
    :param gt_df:
    :return:
    """

    gt_df = gt_df[['id','discourse_type','predictionstring']]\
            .reset_index(drop=True).copy()

    pred_df = pred_df[['id','class','predictionstring']]\
            .reset_index(drop=True).copy()

    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index

    # Step 1. all ground truths and predictions for a given class are compared.

    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how = 'outer',
                           suffixes = ('_pred','_gt')
                           )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')


    joined['overlaps'] = joined.apply(calc_overlap, axis=1)
    #2. If the overlap between the ground truth and prediction is >= 0.5,
    # and the overlap between prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positives
    # if multiple matches exist, the match with the highest pair of overlaps is taken.

    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])

    joined['potential_TP'] = (joined['overlap1'] >= 0.5) &(joined['overlap2']>=0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)

    tp_pred_ids = joined.query('potential_TP') \
            .sort_values('max_overlap', ascending=False) \
            .groupby(['id', 'predictionstring_gt']).first()['pred_id'].values


    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    fn_gt_ids = [g for g in joined['gt_id'].unique() if g not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids )
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    # calc microf1

In [ ]:


d

In [ ]:

for c in CLASSES:
    pred_df = oof.loc[oof['class'] ==c].copy()
    gt_df = valid.loc[valid['discourse_type'] == c ].copy()

    f1 = score_feedback_comp(pred_df, gt_df)

    print(c,f1)
    f1s.append(f1)
print()
print('Overall' , np.mean(f1s))

In [ ]:
preds = [[[1560,1024,15]]]

In [ ]:
np.zeros((len(IDS),MAX_LEN), dtype='int32')

In [266]:
np.zeros((len(IDS),MAX_LEN))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
test_array = np.zeros((len(valid_idx), MAX_LEN, 15),dtype='int32')

In [ ]:
pred = test_array[3,]/2.0 # 2dimension

In [265]:
start = pred[0]     # 1dimension

In [ ]:
word[word_idx+1]

In [ ]:
targets[:,:,2*k+1]

In [ ]:
# FIND TARGETS IN TEXT AND SAVE IN TARGET ARRAYS
# 분류별로 나누어져있는 text를 train set에서 찾기
# 자료 구조도 단순한게 아니라 complex한게 많으니 '이해'를 해야해


In [ ]:
df = train.loc[train.id==n]

In [ ]:
for index, row in df.iterrows():
    break

In [ ]:
text = row['discourse_text']
row['dicourse_type']

In [ ]:
target_map = {'Lead': 0 , 'Position' :1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4, 'CounterClaim':5 , 'Rebuttal' : 6}

In [ ]:
k = target_map[row.discourse_type]

In [ ]:
k

In [ ]:
targets_b[k][id_num][offset_index] = 1

In [ ]:
train[train.id==n]

In [ ]:
IDS[id_num]

In [ ]:
# 해당 offset이 target class안에 속하도록
text

In [ ]:
b

In [ ]:
#offsets[offset_index] 가 a 와 b 사이에 있다면
# 우선 df 에 속하는 게 맞으니까 체크를 시작하는거고
# 얘의 인덱스 start 와 end를 실제 텍스트에서 검사하는 거지


In [ ]:
c =  offsets[offset_index][0]
d =  offsets[offset_index][1]


In [ ]:
c

In [ ]:
df  = train.loc[train.id ==n]
for index, row in df.iterrows():
